# The OpenAI example of function use

In [ ]:
from os.path import dirname
import sys
from dotenv import load_dotenv, find_dotenv
sys.path.append(dirname(find_dotenv()))
load_dotenv()
"""OPENAI_API_KEY or OPENAI_API_KEY_PATH,
OPENAI_ORGANIZATION can be set... or not.
The properly named environment variables are loaded
into the OpenAI library by its' __init__"""
import openai
import json

In [ ]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "69"
                       "",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
available_functions = {
        "get_current_weather": get_current_weather,
}

def execute_function_call(message, functions):
    name = message["function_call"]["name"]
    function_to_call = functions[name]
    arguments = json.loads(response_message["function_call"]["arguments"])
    function_result = function_to_call(
        location=arguments.get("location"),
        unit=arguments.get("unit"),
    )
    return function_result


In [ ]:
# Step 1: send the conversation and available functions to GPT
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Chicago, IL?"
    }
]
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default
)
response_message = response["choices"][0]["message"]

In [ ]:
# Step 2: check if GPT wanted to call a function
if response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors

    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    function_name = response_message["function_call"]["name"]
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = fuction_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    # Step 4: send the info on the function call and function response to GPT
    messages.append(response_message)  # extend conversation with assistant's reply
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

    print(second_response)
else:
    print('There was no function call')
